In [1]:
import sys,os

sys.path.append('/home/arash/ProjectVR/')
#sys.path.append('/home/arash/anaconda3/envs/python36/lib/python3.6/site-packages')

import importlib
import json
import time
import numpy as np
import pandas as pd
import pickle
import shap  # package used to calculate Shap values
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

#import logging
#from logging import handlers
from deepsurv import deep_surv, utils
#from deepsurv.deepsurv_logger import DeepSurvLogger, TensorboardLogger

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood a

In [2]:
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xtrain', 'rb') as f:
    xtrain=pickle.load(f)

network=deep_surv.load_model_from_json(model_fp = '/home/arash/ProjectVR/models/Wait/Deepwait100epochsTuned',
                     weights_fp = '/home/arash/ProjectVR/models/Wait/deepwait_weights_100_epochs.npz')

Loading json model: {"n_in": 19, "learning_rate": 0.0008208386784744157, "hidden_layers_sizes": [90, 90], "lr_decay": 0.000551240234375, "activation": "rectify", "dropout": 0.0, "batch_norm": true, "standardize": false}


In [3]:
#using train set for interpretability
dfxtrain=pd.DataFrame(xtrain[:,0:network.hyperparams['n_in']])
dfxtrain.columns=dfxtrain.columns=['Walk to Shopping', 'More than 1 Car in the Household', 'Main Mode: Car', 
       'Vehicle Arrival Rate', 'Age Over 50',
       'Traffic Density', ' Age: 18 - 29', ' Weather: Snowy', 'Gender: Female', 'Road Type: One Way',
       'Main Mode: Active Modes', 'Participant Owns a Driving License', 'Mixed Traffic Conditions',
       ' Previous VR Experience', 'Age: 30 - 39', 'Fully Automated Conditions', ' Lane Width',
       'Time: Night', 'Walk to Work']

In [4]:
## seperate prev VR experience (reviewer comment)
dfxtrainVR = dfxtrain[dfxtrain[' Previous VR Experience']==1.0] 
dfxtrainNOVR = dfxtrain[dfxtrain[' Previous VR Experience']==0.0] 

In [5]:
#continous variables:
contcol = ['Vehicle Arrival Rate','Traffic Density',' Lane Width']

#using zeros as background data for binary variables, and mean value for cont variables
backgrounddata = pd.DataFrame(np.zeros(network.hyperparams['n_in'])).T  

backgrounddata.columns = dfxtrain.columns
backgrounddata[contcol]=[(dfxtrain['Vehicle Arrival Rate']).mean(),
                         (dfxtrain['Traffic Density']).mean(),
                         (dfxtrain[' Lane Width']).mean()] 

explainer = shap.KernelExplainer(network.predict_risk,backgrounddata)
shap_values = explainer.shap_values(dfxtrain)

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


In [ ]:
fig = shap.summary_plot(shap_values[0], dfxtrain,max_display=20,show =False)
plt.savefig('shap', bbox_inches='tight')

In [ ]:
shap.summary_plot(shap_values[0], dfxtrain, plot_type="bar")


### Create Table of Shap Values for each feature

In [ ]:
def greaterthanmean(col):
    ColMean=col.mean()
    BlueIndex=col[col<=ColMean].index     #indices of feature values less than (or eq to) avg of feature
    return BlueIndex

Blues = {'Vehicle Arrival Rate': greaterthanmean(dfxtrain['Vehicle Arrival Rate']),
         'Traffic Density': greaterthanmean(dfxtrain['Traffic Density']),
        ' Lane Width': greaterthanmean(dfxtrain[' Lane Width'])}


shap_df = pd.DataFrame(shap_values[0], copy = True)
shap_df.columns = dfxtrain.columns
shap_df['Vehicle Arrival Rate'][Blues['Vehicle Arrival Rate']] = \
    shap_df['Vehicle Arrival Rate'][Blues['Vehicle Arrival Rate']]*(-1)
shap_df['Traffic Density'][Blues['Traffic Density']] = \
    shap_df['Traffic Density'][Blues['Traffic Density']]*(-1)
shap_df[' Lane Width'][Blues[' Lane Width']] = shap_df[' Lane Width'][Blues[' Lane Width']]*(-1)





mean=shap_df[shap_df!=0].mean()              #mean of shap values for each feature (non-zeros only

nonzeros=shap_df.astype(bool).sum(axis=0)        #count of non-zeros shap values for each feature

std=shap_df[shap_df!=0].std()                #standard deviation of shap values for each feature

featuresshap=pd.DataFrame()

featuresshap['count']=nonzeros
featuresshap['mean']=mean                 #bg0: zeros set as background data

featuresshap['std']=std

featuresshap['absmean']=abs(mean)

featuresshap.sort_values(by=['absmean'])

In [ ]:
#featuresshap.sort_values(by=['absmean'],ascending=False).drop(columns=['count','absmean']).to_latex(index=False)

In [ ]:
#No VR analysis
explainerVR = shap.KernelExplainer(network.predict_risk,backgrounddata)
shap_valuesVR = explainerVR.shap_values(dfxtrainVR)
shap_valuesNOVR = explainerVR.shap_values(dfxtrainNOVR)

In [ ]:
shapNOVR = np.delete(shap_valuesNOVR[0], 13, 1)
dfxtrainNOVR = dfxtrainNOVR.drop(columns=[' Previous VR Experience'])
shapVR = np.delete(shap_valuesVR[0], 13, 1)
dfxtrainVR = dfxtrainVR.drop(columns=[' Previous VR Experience'])           

In [ ]:
fig = shap.summary_plot(shapNOVR, dfxtrainNOVR,max_display=20,show =False)
plt.savefig('shapNOVR', bbox_inches='tight')


In [ ]:
fig = shap.summary_plot(shapVR, dfxtrainVR,max_display=20,show=False)
plt.savefig('shapVR', bbox_inches='tight')

# Gender

shap.dependence_plot(ind='Vehicle Arrival Rate', interaction_index='Gender: Female', 
                     shap_values=shap_values[0], features=dfxtrain)

shap.dependence_plot(ind='Traffic Density', interaction_index='Gender: Female', 
                     shap_values=shap_values[0], features=dfxtrain)

shap.dependence_plot(ind='Mixed Traffic Conditions', interaction_index='Gender: Female', 
                     shap_values=shap_values[0], features=dfxtrain)

shap.dependence_plot(ind='Fully Automated Conditions', interaction_index='Gender: Female', 
                     shap_values=shap_values[0], features=dfxtrain)

shap.dependence_plot(ind=' Lane Width', interaction_index='Gender: Female', 
                     shap_values=shap_values[0], features=dfxtrain)

shap.dependence_plot(ind='Age Over 50', interaction_index='Gender: Female', 
                     shap_values=shap_values[0], features=dfxtrain)

shap.dependence_plot(ind=' Age: 18 - 29', interaction_index='Gender: Female', 
                     shap_values=shap_values[0], features=dfxtrain)

shap.dependence_plot(ind='Age: 30 - 39', interaction_index='Gender: Female', 
                     shap_values=shap_values[0], features=dfxtrain)

# Age

shap.dependence_plot(ind='Vehicle Arrival Rate', interaction_index='Age Over 50', 
                     shap_values=shap_values[0], features=dfxtrain)

shap.dependence_plot(ind='Traffic Density', interaction_index='Age Over 50', 
                     shap_values=shap_values[0], features=dfxtrain)

shap.dependence_plot(ind=' Weather: Snowy', interaction_index='Age Over 50', 
                     shap_values=shap_values[0], features=dfxtrain)

shap.dependence_plot(ind='Road Type: One Way', interaction_index='Age Over 50', 
                     shap_values=shap_values[0], features=dfxtrain)

shap.dependence_plot(ind='Mixed Traffic Conditions', interaction_index='Age Over 50', 
                     shap_values=shap_values[0], features=dfxtrain)

shap.dependence_plot(ind='Fully Automated Conditions', interaction_index='Age Over 50', 
                     shap_values=shap_values[0], features=dfxtrain)

shap.dependence_plot(ind=' Lane Width', interaction_index='Age Over 50', 
                     shap_values=shap_values[0], features=dfxtrain)

shap.dependence_plot(ind='Time: Night', interaction_index='Age Over 50', 
                     shap_values=shap_values[0], features=dfxtrain)

In [ ]:
shap.dependence_plot(ind='Vehicle Arrival Rate', interaction_index=' Previous VR Experience', shap_values=shap_values[0], features=dfxtrain)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2)
shap.dependence_plot(ind='Traffic Density', interaction_index=' Previous VR Experience', 
                     shap_values=shap_values[0], features=dfxtrain,ax=ax1)

In [ ]:
shap.dependence_plot(ind=var, interaction_index=' Previous VR Experience', 
                     shap_values=shap_values[0], features=dfxtrain,ax=ax4)

In [ ]:
# socio = ['Walk to Shopping',
# 'Walk to Work' , 
# 'More than 1 Car in the Household',
# 'Main Mode: Car', 'Main Mode: Active Modes',
# 'Age Over 50',  ' Age: 18 - 29', 'Age: 30 - 39',
# 'Gender: Female', 
# 'Participant Owns a Driving License',
# ' Previous VR Experience' ]

# exp = ['Vehicle Arrival Rate',
# 'Traffic Density',
# ' Weather: Snowy',
# 'Road Type: One Way',
# 'Mixed Traffic Conditions', 'Fully Automated Conditions',
# ' Lane Width',
# 'Time: Night']

# insig = ['Main Mode: Active Modes','Age: 30 - 39' , 'Age Over 50',
#          'Main Mode: Car', 'Age: 18 - 29', 'More than 1 Car in the Household',
#          'Walk to Shopping', 'Vehicle Arrival Rate', 'Gender: Female' ,
#          'Road Type: One Way', 'Participant Owns a Driving License' ]
# insigind = [0, 1, 2, 3, 4, 6, 8, 9, 10, 11, 14]

inds = shap.approximate_interactions('Main Mode: Active Modes',shap_values[0], dfxtrain )
indsinsig = []
for i in inds:
    if i in insigind:
        indsinsig.append(i)
for i in range(3):
    shap.dependence_plot('Main Mode: Active Modes', shap_values[0], dfxtrain, interaction_index=indsinsig[i])

inds = shap.approximate_interactions('Age Over 50',shap_values[0], dfxtrain )
indsinsig = []
for i in inds:
    if i in insigind:
        indsinsig.append(i)
for i in range(3):
    shap.dependence_plot('Age Over 50', shap_values[0], dfxtrain, interaction_index=indsinsig[i])



inds = shap.approximate_interactions('Main Mode: Car',shap_values[0], dfxtrain )
indsinsig = []
for i in inds:
    if i in insigind:
        indsinsig.append(i)
for i in range(3):
    shap.dependence_plot('Main Mode: Car', shap_values[0], dfxtrain, interaction_index=indsinsig[i])

inds = shap.approximate_interactions('Age: 30 - 39',shap_values[0], dfxtrain )
indsinsig = []
for i in inds:
    if i in insigind:
        indsinsig.append(i)
for i in range(3):
    shap.dependence_plot('Age: 30 - 39', shap_values[0], dfxtrain, interaction_index=indsinsig[i])

inds = shap.approximate_interactions(' Age: 18 - 29',shap_values[0], dfxtrain )
indsinsig = []
for i in inds:
    if i in insigind:
        indsinsig.append(i)
for i in range(3):
    shap.dependence_plot(' Age: 18 - 29', shap_values[0], dfxtrain, interaction_index=indsinsig[i])

inds = shap.approximate_interactions('More than 1 Car in the Household',shap_values[0], dfxtrain )
indsinsig = []
for i in inds:
    if i in insigind:
        indsinsig.append(i)
for i in range(3):
    shap.dependence_plot('More than 1 Car in the Household', shap_values[0], dfxtrain, interaction_index=indsinsig[i])

inds = shap.approximate_interactions('Walk to Shopping',shap_values[0], dfxtrain )
indsinsig = []
for i in inds:
    if i in insigind:
        indsinsig.append(i)
for i in range(3):
    shap.dependence_plot('Walk to Shopping', shap_values[0], dfxtrain, interaction_index=indsinsig[i])

inds = shap.approximate_interactions('Vehicle Arrival Rate',shap_values[0], dfxtrain )
indsinsig = []
for i in inds:
    if i in insigind:
        indsinsig.append(i)
for i in range(3):
    shap.dependence_plot('Vehicle Arrival Rate', shap_values[0], dfxtrain, interaction_index=indsinsig[i])

inds = shap.approximate_interactions('Gender: Female',shap_values[0], dfxtrain )
indsinsig = []
for i in inds:
    if i in insigind:
        indsinsig.append(i)
for i in range(3):
    shap.dependence_plot('Gender: Female', shap_values[0], dfxtrain, interaction_index=indsinsig[i])


inds = shap.approximate_interactions('Road Type: One Way',shap_values[0], dfxtrain )
indsinsig = []
for i in inds:
    if i in insigind:
        indsinsig.append(i)
for i in range(3):
    shap.dependence_plot('Road Type: One Way', shap_values[0], dfxtrain, interaction_index=indsinsig[i])


inds = shap.approximate_interactions('Participant Owns a Driving License',shap_values[0], dfxtrain )
indsinsig = []
for i in inds:
    if i in insigind:
        indsinsig.append(i)
for i in range(3):
    shap.dependence_plot('Participant Owns a Driving License', shap_values[0], dfxtrain, interaction_index=indsinsig[i])

In [18]:
shapdf = pd.DataFrame((shap_values[0]),columns = dfxtrain.columns)


In [23]:
#continous variables:
contcol = ['Vehicle Arrival Rate','Traffic Density',' Lane Width']

In [24]:
#using zeros as background data for binary variables, and mean value for cont variables
backgrounddata=pd.DataFrame(np.zeros(19)).T  

In [25]:
backgrounddata.columns = shapdf.columns

In [26]:
backgrounddata[contcol]=[(dfxtrain['Vehicle Arrival Rate']).mean(),
                         (dfxtrain['Traffic Density']).mean(),
                         (dfxtrain[' Lane Width']).mean()] 

In [47]:
def interaction(a,b):
    inter = shapdf[a][(dfxtrain[b] > backgrounddata[b][0]) &
                      (dfxtrain[a] !=  backgrounddata[a][0])]
    return inter


In [48]:
names = shapdf.columns 

In [49]:
MEANintersshap = pd.DataFrame(index=range(19), columns=range(19))
STDintersshap = pd.DataFrame(index=range(19), columns=range(19))

n = 0
for i in names:
    m = 0
    for j in names:
        MEANintersshap.rename(index={n: i}, columns={m: j}, inplace=True)
        STDintersshap.rename(index={n: i}, columns={m: j}, inplace=True)
        if (i != j):
            a = i
            b = j
            MEANintersshap.iloc[n,m] = interaction(i, j).mean()
            STDintersshap.iloc[n,m] = interaction(i, j).std()
        m = m + 1
    n = n + 1
        
            #std= interaction(a, b).std() 
            #stds.append(std)

In [ ]:
# def interactionplus(a,b,c):
#     inter = shapdf[a][(dfxtrain[c] != 1.0) & (dfxtrain[b] != 1.0) & (dfxtrain[a] != 0.0)]
#     return inter

In [ ]:
# MEANFAV = pd.DataFrame(index=range(19), columns=range(1))
# STDFAV = pd.DataFrame(index=range(19), columns=range(1))

# n = 0
# for i in names:
#     m = 0
#     j = 'Fully Automated Conditions'
#     MEANFAV.rename(index={n: i}, columns={m: j}, inplace=True)
#     STDFAV.rename(index={n: i}, columns={m: j}, inplace=True)
#     if (i != j):
#         a = i
#         b = j
#         MEANFAV.iloc[n,m] = interactionplus(i, j,'Mixed Traffic Conditions').mean()
#         STDFAV.iloc[n,m] = interactionplus(i, j,'Mixed Traffic Conditions').std()
        
#     n = n + 1
        
#             #std= interaction(a, b).std() 
#             #stds.append(std)

In [ ]:
# MEANHDV = pd.DataFrame(index=range(19), columns=range(1))
# STDHDV = pd.DataFrame(index=range(19), columns=range(1))

# n = 0
# for i in names:
#     m = 0
#     j = 'Human Driven Condition'
#     MEANHDV.rename(index={n: i}, columns={m: j}, inplace=True)
#     STDHDV.rename(index={n: i}, columns={m: j}, inplace=True)
#     if (i != j):
#         a = i
#         b = j
#         MEANHDV.iloc[n,m] = interactionplus(i, 'Fully Automated Conditions', 'Mixed Traffic Conditions').mean()
#         STDHDV.iloc[n,m] = interactionplus(i, 'Fully Automated Conditions', 'Mixed Traffic Conditions').std()
        
#     n = n + 1
        
#             #std= interaction(a, b).std() 
#             #stds.append(std)

In [ ]:
# pd.concat([MEANHDV,STDHDV],axis=1)

In [ ]:
# pd.concat([MEANFAV,STDFAV],axis=1)

In [ ]:
# shapFAVmax = MEANFAV + STDFAV
# shapHDVmax = MEANHDV + STDHDV
# shapFAVmin = MEANFAV - STDFAV
# shapHDVmin = MEANHDV - STDHDV

In [50]:
shapintmax = MEANintersshap + STDintersshap

In [51]:
shapintmin = MEANintersshap - STDintersshap

In [52]:
signint = shapintmin * shapintmax
#signFAV = shapFAVmin * shapFAVmax
#signHDV = shapHDVmin * shapHDVmax

In [ ]:
# MEANFAV[signFAV>0]

In [ ]:
# MEANHDV[signHDV>0]

In [53]:
xxx = MEANintersshap[signint>0].stack()

In [56]:
MEANintersshap

,Walk to Shopping,More than 1 Car in the Household,Main Mode: Car,Vehicle Arrival Rate,Age Over 50,Traffic Density,Age: 18 - 29,Weather: Snowy,Gender: Female,Road Type: One Way,Main Mode: Active Modes,Participant Owns a Driving License,Mixed Traffic Conditions,Previous VR Experience,Age: 30 - 39,Fully Automated Conditions,Lane Width,Time: Night,Walk to Work
Walk to Shopping,NaN,0.119942,0.000914904,-0.150458,-0.436315,-0.130957,-0.0237993,-0.109257,-0.041964,0.0325042,0.0551756,-0.0569535,0.0308174,-0.0170937,-0.0475614,-0.082234,-0.046916,-0.0168941,-0.00590971
More than 1 Car in the Household,0.202083,NaN,0.254095,0.162741,0.296959,0.134229,0.193788,0.147381,0.162107,0.213359,0.00474888,0.124851,0.437187,0.196081,-0.101623,0.119745,0.0712245,0.111016,0.246243
Main Mode: Car,-0.176816,-0.180832,NaN,-0.329464,-0.299242,-0.289585,-0.377411,-0.207784,-0.497206,-0.179756,NaN,-0.219275,-0.18765,-0.230479,-0.0762915,-0.225448,-0.214446,-0.243203,-0.339106
Vehicle Arrival Rate,0.0153633,0.0202896,0.0219487,NaN,0.0349775,0.0658144,0.0184682,0.0160545,0.0117118,0.00978967,0.00148642,0.0176637,0.043969,0.0177262,0.0124169,0.0186456,0.0159227,0.010965,0.0125813
Age Over 50,-0.3051,-0.359485,-0.409841,-0.289521,NaN,-0.211837,NaN,-0.297813,-0.385707,-0.170936,NaN,-0.224042,-0.262246,-0.590341,NaN,-0.219882,-0.179658,-0.199592,-0.325574
Traffic Density,0.0328286,0.0212439,0.0287359,-0.198388,0.0250157,NaN,0.0296675,0.00388961,0.0257604,0.0393481,0.0167018,0.0337588,0.0542891,0.0120806,0.0357722,0.0271215,0.0423925,0.0497529,0.025301
Age: 18 - 29,-0.113665,-0.190328,-0.341791,-0.192974,NaN,-0.18379,NaN,-0.0702391,-0.301551,-0.184256,-0.203549,-0.148451,-0.30076,-0.178674,NaN,-0.142462,-0.0678149,-0.172604,-0.244236
Weather: Snowy,-0.256184,-0.188363,-0.176761,-0.302445,-0.377821,-0.275998,-0.161746,NaN,-0.197193,-0.279479,-0.26829,-0.230172,0.115462,-0.208314,-0.277545,-0.250668,-0.154306,-0.23487,-0.212731
Gender: Female,0.0408782,-0.0700838,-0.264118,-0.0669686,0.000876433,-0.0259598,-0.112196,0.0394808,NaN,-0.0229413,0.0260471,0.0354777,-0.100891,0.0149818,0.219342,0.0500531,0.0231309,0.025047,-0.0187079
Road Type: One Way,-0.0374965,0.035739,0.0698737,0.0532569,0.130293,0.0568819,-0.0745502,-0.0721916,-0.0832846,NaN,-0.110418,-0.00975516,0.123588,-0.0545671,0.0206206,-0.0361233,-0.0226576,-0.0932206,-0.116363


In [54]:
xxx.to_csv('interactionsig.csv')

In [ ]:
MEANVR = pd.DataFrame(index=range(1), columns=range(19))
STDVR = pd.DataFrame(index=range(1), columns=range(19))

i = ' Previous VR Experience'
m = 0
for j in names:
    MEANVR.rename(index={0: i}, columns={m: j}, inplace=True)
    STDVR.rename(index={0: i}, columns={m: j}, inplace=True)
    if (i != j):
        a = i
        b = j
        MEANVR.iloc[0,m] = interaction(i, j).mean()
        STDVR.iloc[0,m] = interaction(i, j).std()
    m = m + 1

In [ ]:
MEANVR

In [ ]:
interaction(' Previous VR Experience', names[18]).mean()